In [0]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, desc

# Create my_spark
my_spark = SparkSession.builder.appName("my_spark").getOrCreate()

# Print my_spark
print(my_spark)

In [0]:
myRange = my_spark.range(1000).toDF('number')
myRange.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [0]:
divideBy2 = myRange.where("number % 2 = 0")
divideBy2.show(5)

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
+------+
only showing top 5 rows



In [0]:
divideBy2.count()

Out[4]: 500

In [0]:
flightData2010 = spark\
    .read\
        .option('header', 'true')\
            .option('inferSchema', 'true')\
                .csv("/FileStore/tables/2010_summary.csv")

In [0]:
flightData2010.take(3)

Out[6]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69)]

In [0]:
flightData2010.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [0]:
flightData2010.sort('count').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#651 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#651 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=879]
      +- FileScan csv [DEST_COUNTRY_NAME#649,ORIGIN_COUNTRY_NAME#650,count#651] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/2010_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [0]:
my_spark.conf.set('spark.sql.shuffle.partitions', '5')
flightData2010.sort('count').take(2)

Out[9]: [Row(DEST_COUNTRY_NAME='Equatorial Guinea', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1)]

In [0]:
flightData2010.createOrReplaceTempView('flight_data_2010')

In [0]:
sql_way = my_spark.sql("""SELECT DEST_COUNTRY_NAME, COUNT(1)
                       FROM FLIGHT_DATA_2010
                       GROUP BY DEST_COUNTRY_NAME""")
dataFrameWay = flightData2010\
    .groupby('DEST_COUNTRY_NAME')\
        .count()

sql_way.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#649], functions=[finalmerge_count(merge count#690L) AS count(1)#678L])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#649, 5), ENSURE_REQUIREMENTS, [plan_id=909]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#649], functions=[partial_count(1) AS count#690L])
         +- FileScan csv [DEST_COUNTRY_NAME#649] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/2010_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#649], functions=[finalmerge_count(merge count#692L) AS count(1)#685L])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#649, 5), ENSURE_REQUIREMENTS, [plan_id=930]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#649], functions=[partial_count(1) AS count#692L])
         +- FileScan

In [0]:
my_spark.sql("SELECT MAX(COUNT) FROM FLIGHT_DATA_2010").take(1)
flightData2010.select(max('count')).take(1)

Out[20]: [Row(max(count)=348113)]

In [0]:
maxSql = my_spark.sql("""SELECT DEST_COUNTRY_NAME, SUM(COUNT) AS DESTINATION_TOTAL
                      FROM FLIGHT_DATA_2010
                      GROUP BY DEST_COUNTRY_NAME
                      ORDER BY SUM(COUNT) DESC
                      LIMIT 5;""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|DESTINATION_TOTAL|
+-----------------+-----------------+
|    United States|           384932|
|           Canada|             8271|
|           Mexico|             6200|
|   United Kingdom|             1629|
|          Germany|             1392|
+-----------------+-----------------+



In [0]:
flightData2010\
    .groupby('DEST_COUNTRY_NAME')\
        .sum('count')\
            .withColumnRenamed('sum(count)', 'destination_total')\
                .sort(desc('destination_total'))\
                    .limit(5)\
                        .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           384932|
|           Canada|             8271|
|           Mexico|             6200|
|   United Kingdom|             1629|
|          Germany|             1392|
+-----------------+-----------------+



In [0]:
flightData2010\
    .groupby('DEST_COUNTRY_NAME')\
        .sum('count')\
            .withColumnRenamed('sum(count)', 'destination_total')\
                .sort(desc('destination_total'))\
                    .limit(5)\
                        .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#876L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#649,destination_total#876L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#649], functions=[finalmerge_sum(merge sum#880L) AS sum(count#651)#872L])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#649, 5), ENSURE_REQUIREMENTS, [plan_id=1340]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#649], functions=[partial_sum(count#651) AS sum#880L])
            +- FileScan csv [DEST_COUNTRY_NAME#649,count#651] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/2010_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


